In [ ]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.chrome.options import Options

In [ ]:
#get the urls of the first n pages
n = 10
page_list = ['https://www.curseforge.com/minecraft/modpacks?page=' + str(b+1) for b in range(n)]
page_list

In [ ]:
#get the urls of every modpack on the first n pages
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
modpacks = []
for url in page_list:
    driver.get(url)
    for modpack in driver.find_elements(by=By.CLASS_NAME, value='project-listing-row'):
        modpack_url = modpack.find_elements(by=By.CLASS_NAME, value='my-auto')[0].get_attribute('href')
        modpacks.append(modpack_url)
driver.quit()
modpacks

In [ ]:
#for each modpack, get the number of downloads, version number, and number of pages needed to list the dependencies
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

modpack_mods = []
for url in modpacks:
    driver.get(url + '/relations/dependencies')
    game_version = driver.find_elements(by=By.CLASS_NAME, value='text-gray-500')[2].text.split(' ')[2]
    downloads = driver.find_elements(by=By.CLASS_NAME, value='text-gray-500')[0].text.split(' ')[0]
    page_numbers = driver.find_elements(by=By.CLASS_NAME, value='pagination-top')
    if len(page_numbers) == 0:
        num_pages = 1
    else:
        num_pages = int(page_numbers[0].find_elements(by=By.CLASS_NAME, value='text-primary-500')[-1].text)
    mods = []
    modpack_mods.append({'url':url, 'game_version':game_version, 'total_downloads':downloads, 'num_pages':num_pages})
driver.close()

modpack_mods_df = pd.DataFrame(modpack_mods)
modpack_mods_df

In [ ]:
modpack_mods_df['game_version'].value_counts().plot(kind="bar")

In [ ]:
#find a list of every mod in each modpack (takes a long time)
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

for modpack in modpack_mods:
    pages = [modpack['url'] + '/relations/dependencies?page=' + str(b+1) for b in range(modpack['num_pages'])]
    mods = []
    for page in pages:
        driver.get(page)
        mods.extend([b.find_elements(by=By.TAG_NAME, value='a')[0].get_attribute('href') for b in driver.find_elements(by=By.CLASS_NAME, value='project-listing-row')])
    #print(mods)
    modpack.update({'mods':mods})

driver.close()

modpack_mods_df = pd.DataFrame(modpack_mods)
modpack_mods_df

In [ ]:
modpack_mods_df['mods'].mask(modpack_mods_df['mods'].isna(), 'incomplete').where(modpack_mods_df['mods'].isna(), 'complete').value_counts().plot(kind="bar")

In [ ]:
modpack_mods_df.to_csv('modpacks.csv')

In [ ]:
mod_url = ['https://www.curseforge.com/minecraft/mc-mods/kubejs', 'https://www.curseforge.com/minecraft/mc-mods/mekanism', 'https://www.curseforge.com/minecraft/mc-mods/pneumaticcraft-repressurized']
[b['url'] + ' version: ' + b['game_version']  for b in modpack_mods if 'mods' in b.keys() and set(mod_url) == set(mod_url) & set(b['mods'])]
